In [332]:
import os
import time
import pandas as pd
import simplekml
import shutil
from sys import platform
from pathlib import Path
pd.options.mode.chained_assignment = None

# Measuring execution time
start_time = time.time()

In [333]:
# Checks what operating system the user is using
if platform == 'win32':
    root_path = r'C:\Users\Stefanos\DeLasCasas CP, LLc\Ro De Las Casas - ' \
                r'DLC2P Server\AA Stefanos\Data Science\Python Projects\CIS ' \
                r'Data to Google Earth (Real Time)'
elif platform == 'darwin':
    root_path = r'/Users/Stefanos/Library/CloudStorage/' \
                r'OneDrive-SharedLibraries-DeLasCasasCP,LLc/Ro De Las Casas ' \
                r'- DLC2P Server/AA Stefanos/Data Science/Python Projects/' \
                r'CIS Data to Google Earth (Real Time)'

# Specify Variables

In [334]:
# SPECIFY SCALE FACTOR
scale_factor = 500
# SPECIFY TRANSPARENCY (0 to 1)
transparency = 1
# SPECIFY ICON SCALE
icon_scale = 0.2
# SPECIFY COLOR PALETTE
#my_cmap = plt.get_cmap('Spectral_r')
# PROJECT NAME
project_name = 'XYZ'    #input('\nPROJECT NAME:')

In [337]:
from math import atan
from math import atan2
from math import cos
from math import radians
from math import asin
from math import sin
from math import sqrt
from math import tan

In [338]:
class VincentyInverse:
    def __init__(self, coord1, coord2, maxiter=200, tol=10**-12):

        #--- CONSTANTS ------------------------------------+

        a = 6378137.0                    # radius at equator in meters (WGS-84)
        f = 1/298.257223563              # flattening of the ellipsoid (WGS-84)
        b = (1-f)*a

        phi_1, lon_1, = coord1                      # (lat=L_?,lon=phi_?)
        phi_2, lon_2, = coord2

        u_1 = atan((1-f)*tan(radians(phi_1)))
        u_2 = atan((1-f)*tan(radians(phi_2)))

        L = radians(lon_2-lon_1)

        Lambda = L                           # set initial value of lambda to L

        sin_u1 = sin(u_1)
        cos_u1 = cos(u_1)
        sin_u2 = sin(u_2)
        cos_u2 = cos(u_2)

        #--- BEGIN ITERATIONS -----------------------------+
        self.iters = 0
        for i in range(0, maxiter):
            self.iters += 1

            cos_lambda = cos(Lambda)
            sin_lambda = sin(Lambda)
            sin_sigma = sqrt((cos_u2*sin(Lambda))**2+
                             (cos_u1*sin_u2-sin_u1*cos_u2*cos_lambda)**2)
            cos_sigma = sin_u1*sin_u2+cos_u1*cos_u2*cos_lambda
            sigma = atan2(sin_sigma,cos_sigma)
            sin_alpha = (cos_u1*cos_u2*sin_lambda)/sin_sigma
            cos_sq_alpha = 1-sin_alpha**2
            cos2_sigma_m = cos_sigma-((2*sin_u1*sin_u2)/cos_sq_alpha)
            C = (f/16)*cos_sq_alpha*(4+f*(4-3*cos_sq_alpha))
            Lambda_prev = Lambda
            Lambda = L+(1-C)*f*sin_alpha*(sigma+C*sin_sigma*
                                          (cos2_sigma_m+C*cos_sigma*(-1+2*cos2_sigma_m**2)))

            # Successful convergence
            diff = abs(Lambda_prev-Lambda)
            if diff <= tol:
                break

        u_sq = cos_sq_alpha*((a**2-b**2)/b**2)
        A = 1+(u_sq/16384)*(4096+u_sq*(-768+u_sq*(320-175*u_sq)))
        B = (u_sq/1024)*(256+u_sq*(-128+u_sq*(74-47*u_sq)))
        delta_sig = B*sin_sigma*(cos2_sigma_m+0.25*B*
                                 (cos_sigma*(-1+2*cos2_sigma_m**2)-
                                  (1/6)*B*cos2_sigma_m*(-3+4*sin_sigma**2)*
                                  (-3+4*cos2_sigma_m**2)))

        self.m = b*A*(sigma-delta_sig)         # output distance in meters
        #self.km = self.m/1000                 # output distance in kilometers
        #self.mm = self.m*1000                 # output distance in millimeters
        self.miles = self.m*0.000621371        # output distance in miles
        self.ft = self.miles*5280              # output distance in feet
        #self.inches = self.ft*12              # output distance in inches
        #self.yards = self.ft/3                # output distance in yards

# if __name__ == "__VincentyInverse__":
#     main()

# Data Cleaning

In [340]:
# Creates a list of files in data folder
raw_data_list = os.listdir(r'/Users/Stefanos/Library/CloudStorage/'
                           r'OneDrive-SharedLibraries-DeLasCasasCP,LLc/'
                           r'Ro De Las Casas - DLC2P Server/AA Stefanos/'
                           r'Data Science/Python Projects/'
                           r'CIS Data to Google Earth (Real Time)/'
                           r'Original Data')

In [341]:
# Create an empty list to exclude certain files
filtered_data_list = []
# Current list index
i = 0

# Filters the folder and excludes files that start with '.'
while i < len(raw_data_list):
    if not raw_data_list[i][0] == '.':
        filtered_data_list.append(raw_data_list[i])
    i += 1

In [342]:
# Creates an empty list to place the total miles
total_miles_list = []
# Current list index
i = 0
j = 0

# Goes through all the files in the list
while i < len(filtered_data_list):
    # Create dataframe from a csv file
    df_cis = pd.read_csv(root_path +
                         str(Path(r'/Original Data/' +
                                  filtered_data_list[i])))
    # Drops elevation and PDOP columns
    df_cis = df_cis.iloc[:, :6]
    # Delete rows that CIS was skipped
    df_cis = df_cis.loc[df_cis['On Potential'] != 'SKIP'].reset_index(drop=True)
    # Convert object columns to numbers
    df_cis[['On Potential', 'Off Potential']] = \
                            df_cis[['On Potential', 'Off Potential']]\
                            .apply(pd.to_numeric, errors='coerce', axis=1)
    df_cis[['On Potential', 'Off Potential']] /= 1000
    last_index = df_cis.last_valid_index()
    df_cis['Distance (ft)'] = 0
    while j < last_index:
        try:
            df_cis['Distance (ft)'][j+1] = \
            VincentyInverse([df_cis['Latitude'][j],
                             df_cis['Longitude'][j]],
                            [df_cis['Latitude'][j+1],
                             df_cis['Longitude'][j+1]]).ft
        except ZeroDivisionError:
            df_cis['Distance (ft)'][j+1] = 0

        j += 1

    # Records total miles in a list
    total_miles_list.append(round(max(df_cis['Station'])/5280, 2))
    df_cis.to_csv(root_path +
                  str(Path(r'/Data/' +
                           filtered_data_list[i])), index=False)

    i += 1
    j = 0

# Create Directories (Folders)

In [343]:
# Delete old output folder and create a new one
if os.path.exists(root_path + r'/Output/'):
    shutil.rmtree(root_path + r'/Output/')
    os.makedirs(root_path + r'/Output/')
if not os.path.exists(root_path + r'/Output/'):
    os.makedirs(root_path + r'/Output/')

In [344]:
folder_mile = 1
i = 0

# Create CIS survey folders
while i < len(total_miles_list):
    # miles_remaining = total_miles_list[i]

    # while miles_remaining > 0:
    os.makedirs(root_path + r'/Output/' +
                filtered_data_list[i].split('.DAT', 1)[0] + '/')
        # folder_mile += 1
        # miles_remaining -= 1
        #
        # if 0 < miles_remaining < 1:
        #     folder_mile = total_miles_list[i]

    i += 1
    #folder_mile = 1

# CIS Data

## On

In [345]:
feet_counter = 5280
kml_mile = 1
i = 0       # Row for data
j = 0       # Row for CIS surveys

# Goes through all CIS survey files
while j < len(filtered_data_list):
    df_cis_on = pd.read_csv(root_path + r'/Data/' + filtered_data_list[j])
    df_cis_on = df_cis_on[df_cis_on['On Potential'] != 0]
    df_cis_on = df_cis_on[['Station', 'Longitude', 'Latitude', 'On Potential']]\
        .reset_index(drop=True)
    df_cis_on['On Potential'] = df_cis_on['On Potential']*(-1)
    last_index = df_cis_on.last_valid_index()
    miles_remaining = total_miles_list[j]

    # Create '.kml' files for each CIS survey
    while miles_remaining > 0:
        kml = simplekml.Kml()
        style = simplekml.Style()
        # Create 3D data points
        while i <= last_index and df_cis_on.loc[i, 'Station'] < feet_counter:
            pnt = kml.newpoint(name=df_cis_on.loc[i, 'On Potential']*(-1))
            pnt.style.balloonstyle.text = \
                'Potential: -{} V'.format(df_cis_on.loc[i, 'On Potential'])
            pnt.coords = [(df_cis_on.loc[i, 'Longitude'],
                           df_cis_on.loc[i, 'Latitude'],
                           df_cis_on.loc[i, 'On Potential']*scale_factor)]
            pnt.altitudemode = simplekml.AltitudeMode.relativetoground
            pnt.style.iconstyle.icon.href = \
                'https://img.icons8.com/material-sharp/24/40C057/give-way--v1.png'
            pnt.style.iconstyle.scale = icon_scale
            pnt.style.labelstyle.scale = 0
            pnt.extrude = 0
            i += 1

        kml.save(root_path + r'/Output/' +
             filtered_data_list[j].split('.DAT', 1)[0] + '/' +
             'Mile ' + kml_mile.__str__() + ' - ' + 'On.kml')
        kml_mile += 1
        miles_remaining -= 1

        if 0 < miles_remaining < 1:
            kml_mile = total_miles_list[j]

        feet_counter += 5280

    # Reset variables for next CIS survey
    feet_counter = 5280
    kml_mile = 1
    i = 0
    j += 1

## Off

In [346]:
#df = df_cis_on[df_cis_on.duplicated(subset=['Longitude', 'Latitude'],
                                    #keep=False)]

In [348]:
feet_counter = 5280
kml_mile = 1
i = 0       # Row for data
j = 0       # Row for CIS surveys

# Goes through all CIS survey files
while j < len(filtered_data_list):
    df_cis_off = pd.read_csv(root_path + r'/Data/' + filtered_data_list[j])
    df_cis_off = df_cis_off[df_cis_off['Off Potential'] != 0]
    df_cis_off = df_cis_off[['Station', 'Longitude', 'Latitude',
                           'Off Potential']].reset_index(drop=True)
    df_cis_off['Off Potential'] = df_cis_off['Off Potential']*(-1)
    last_index = df_cis_off.last_valid_index()
    miles_remaining = total_miles_list[j]

    # Create '.kml' files for each CIS survey
    while miles_remaining > 0:
        kml = simplekml.Kml()
        style = simplekml.Style()
        # Create 3D data points
        while i <= last_index and df_cis_off.loc[i, 'Station'] < feet_counter:
            pnt = kml.newpoint(name=df_cis_off.loc[i, 'Off Potential']*(-1))
            pnt.style.balloonstyle.text = \
                'Potential: -{} V'.format(df_cis_off.loc[i, 'Off Potential'])
            pnt.coords = [(df_cis_off.loc[i, 'Longitude'],
                           df_cis_off.loc[i, 'Latitude'],
                           df_cis_off.loc[i, 'Off Potential']*scale_factor)]
            pnt.altitudemode = simplekml.AltitudeMode.relativetoground
            pnt.style.iconstyle.icon.href = \
                'https://img.icons8.com/material-sharp/24/FA5252/give-way--v1.png'
            pnt.style.iconstyle.scale = icon_scale
            pnt.style.labelstyle.scale = 0
            pnt.extrude = 0
            i += 1

        kml.save(root_path + r'/Output/' +
             filtered_data_list[j].split('.DAT', 1)[0] + '/' +
             'Mile ' + kml_mile.__str__() + ' - ' + 'Off.kml')
        kml_mile += 1
        miles_remaining -= 1

        if 0 < miles_remaining < 1:
            kml_mile = total_miles_list[j]

        feet_counter += 5280

    # Reset variables for next CIS survey
    feet_counter = 5280
    kml_mile = 1
    i = 0
    j += 1

## -1.2 V

In [352]:
feet_counter = 5280
kml_mile = 1
i = 0       # Row for data
j = 0       # Row for CIS surveys

# Goes through all CIS survey files
while j < len(filtered_data_list):
    df_cis_1200 = pd.read_csv(root_path + r'/Data/' + filtered_data_list[j])
    df_cis_1200['-1.2 V'] = 1.2
    last_index = df_cis_1200.last_valid_index()
    miles_remaining = total_miles_list[j]

    # Create '.kml' files for each CIS survey
    while miles_remaining > 0:
        kml = simplekml.Kml()
        style = simplekml.Style()
        # Create 3D data points
        while i < last_index and df_cis_1200.loc[i, 'Station'] < feet_counter:
            ls = kml.newlinestring()
            ls.style.balloonstyle.text = 'Potential: -1.2 V'
            ls.coords = [(df_cis_1200.loc[i, 'Longitude'],
                          df_cis_1200.loc[i, 'Latitude'],
                          df_cis_1200.loc[i, '-1.2 V']*scale_factor),
                          (df_cis_1200.loc[i+1, 'Longitude'],
                           df_cis_1200.loc[i+1, 'Latitude'],
                           df_cis_1200.loc[i+1, '-1.2 V']*scale_factor)]
            ls.altitudemode = simplekml.AltitudeMode.relativetoground
            ls.style.linestyle.width = 5
            ls.style.linestyle.color = simplekml.Color.rgb(175, 175, 175, 255)
            i += 1

        kml.save(root_path + r'/Output/' +
             filtered_data_list[j].split('.DAT', 1)[0] + '/' +
             'Mile ' + kml_mile.__str__() + ' - ' + '-1.2 V.kml')
        kml_mile += 1
        miles_remaining -= 1

        if 0 < miles_remaining < 1:
            kml_mile = total_miles_list[j]

        feet_counter += 5280

    # Reset variables for next CIS survey
    feet_counter = 5280
    kml_mile = 1
    i = 0
    j += 1

## -0.85 V

In [353]:
feet_counter = 5280
kml_mile = 1
i = 0       # Row for data
j = 0       # Row for CIS surveys

# Goes through all CIS survey files
while j < len(filtered_data_list):
    df_cis_850 = pd.read_csv(root_path + r'/Data/' + filtered_data_list[j])
    df_cis_850['-0.85 V'] = 0.85
    last_index = df_cis_850.last_valid_index()
    miles_remaining = total_miles_list[j]

    # Create '.kml' files for each CIS survey
    while miles_remaining > 0:
        kml = simplekml.Kml()
        style = simplekml.Style()
        # Create 3D data points
        while i < last_index and df_cis_850.loc[i, 'Station'] < feet_counter:
            ls = kml.newlinestring()
            ls.style.balloonstyle.text = 'Potential: -0.85 V'
            ls.coords = [(df_cis_850.loc[i, 'Longitude'],
                          df_cis_850.loc[i, 'Latitude'],
                          df_cis_850.loc[i, '-0.85 V']*scale_factor),
                          (df_cis_850.loc[i+1, 'Longitude'],
                           df_cis_850.loc[i+1, 'Latitude'],
                           df_cis_850.loc[i+1, '-0.85 V']*scale_factor)]
            ls.altitudemode = simplekml.AltitudeMode.relativetoground
            ls.style.linestyle.width = 5
            ls.style.linestyle.color = simplekml.Color.rgb(175, 175, 175, 255)
            i += 1

        kml.save(root_path + r'/Output/' +
             filtered_data_list[j].split('.DAT', 1)[0] + '/' +
             'Mile ' + kml_mile.__str__() + ' - ' + '-0.85 V.kml')
        kml_mile += 1
        miles_remaining -= 1

        if 0 < miles_remaining < 1:
            kml_mile = total_miles_list[j]

        feet_counter += 5280

    # Reset variables for next CIS survey
    feet_counter = 5280
    kml_mile = 1
    i = 0
    j += 1

## Pipe

In [ ]:
kml = simplekml.Kml()

In [ ]:
df_cis_pipe = df_cis.iloc[:, [False, False, False, False, True, True, False]].copy()
df_cis_pipe['Potential (V)'] = 0

In [ ]:
# Counters
i = 0                                    # Row
z = 282                                  # Max iterations

while i < z:
    ls = kml.newlinestring()
    ls.style.balloonstyle.text = 'Ground'
    ls.coords = [(df_cis_pipe.loc[i, 'LONGITUDE'], df_cis_pipe.loc[i, 'LATITUDE'], df_cis_pipe.loc[i, 'Potential (V)']),
                 (df_cis_pipe.loc[i+1, 'LONGITUDE'], df_cis_pipe.loc[i+1, 'LATITUDE'], df_cis_pipe.loc[i+1, 'Potential (V)'])]

    # Style
    ls.style.linestyle.width = 5
    ls.style.linestyle.color = simplekml.Color.rgb(255, 255, 255, 175)

    i += 1

kml.save(root_path + r'\Output/' + 'Ground - ' + project_name + '.kml')

## Comments

In [ ]:
kml = simplekml.Kml()

In [ ]:
df_cis_comments = df_cis.iloc[:, [False, False, False, True, True, True, False]].copy()

In [ ]:
# Counters
i = 0                                  # Row
z = df_cis_comments.last_valid_index()       # Max iterations

while i < z:
    # noinspection SpellCheckingInspection
    if pd.isna(df_cis_comments.loc[i, 'COMMENTS']):
        i += 1

    else:
        pnt = kml.newpoint(name=df_cis_comments.loc[i, 'COMMENTS'])
        pnt.coords = [(df_cis_comments.loc[i, 'LONGITUDE'], df_cis_comments.loc[i, 'LATITUDE'], 0)]

        pnt.altitudemode = simplekml.AltitudeMode.relativetoground

        # Style
        pnt.style.iconstyle.icon.href = 'https://maps.google.com/mapfiles/kml/paddle/ltblu-blank.png'
        pnt.style.iconstyle.scale = icon_scale*0.05
        #pnt.style.labelstyle.scale = 0

        i += 1

kml.save(root_path + r'\Output/' + 'Comments - ' + project_name + '.kml')